In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import torchvision
import librosa as lb
import torch.nn as nn
from torchvision.models import efficientnet_v2_m

class CFG():
    def __init__(self):
        self.device = torch.device("cpu")  # Csak CPU használata
        self.sample_rate = 32000          # Mintavételi frekvencia
        self.n_mels = 224                 # Mel sávok száma
        self.n_fft = 2048                 # FFT ablakhossz
        self.hop_length = 512             # FFT lépésköz
        self.max_length_s = 5             # Szeletek hossza másodpercben
        self.f_max = 16000                # Maximális frekvencia
        self.f_min = 20                   # Minimális frekvencia
        self.num_classes = 182            # Osztályok száma
        self.model_path = "model.pth"     # Modell mentett állapota
        self.data_path = "data/unlabeled_soundscapes/"  # Hangfájlok helye
        self.output_path = "submission.csv"  # Kimeneti fájl neve

config = CFG()

model = efficientnet_v2_m()
model.classifier[1] = nn.Linear(
    model.classifier[1].in_features, config.num_classes
)
# model.load_state_dict(torch.load(config.model_path, map_location=config.device))
model.eval()
model.to(config.device)


In [32]:
# Fájlok listázása
def list_audio_files(data_dir):
    """
    Listázza az összes audiofájlt a megadott mappában.
    """
    filepaths = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith(".ogg")]
    return pd.DataFrame({"filepath": filepaths})

# Audio szeletelése
def process_audio(file_path):
    """
    Betölti az audiofájlt és 5 másodperces szeletekre bontja.
    """
    audio, _ = lb.load(file_path, sr=config.sample_rate, mono=True)
    chunk_size = config.sample_rate * config.max_length_s
    chunks = [audio[i:i+chunk_size] for i in range(0, len(audio), chunk_size)]
    return chunks

def mel_spectrogram(chunk):
    mel_spec = lb.feature.melspectrogram(
        y=chunk, sr=config.sample_rate, n_fft=config.n_fft,
        hop_length=config.hop_length, n_mels=config.n_mels,
        fmin=config.f_min, fmax=config.f_max
    )
    mel_spec_db = lb.power_to_db(mel_spec, ref=np.max)
    
    # Három csatornás bővítés
    mel_spec_3ch = np.repeat(mel_spec_db[np.newaxis, :, :], 3, axis=0)  # [3, height, width]
    return mel_spec_3ch


In [33]:
def generate_predictions(chunks, model, config):
    predictions = []
    for chunk in chunks:
        mel_spec = mel_spectrogram(chunk)
        
        # Átméretezés az EfficientNet bemeneti követelményeinek megfelelően
        mel_tensor = torch.tensor(mel_spec, dtype=torch.float32).unsqueeze(0).to(config.device)  # [1, 3, height, width]
        
        with torch.no_grad():
            pred = model(mel_tensor)
            predictions.append(pred.cpu().numpy())
    return predictions

# Eredmények mentése CSV-be
def save_predictions_to_csv(filename, predictions):
    """
    A predikciók mentése Kaggle submission formátumban.
    """
    # Predikciók átalakítása [48, 182] alakú mátrixszá
    predictions = np.squeeze(np.array(predictions), axis=1)  # [48, 182]

    # Row ID generálása
    row_ids = [f"soundscape_{filename}_{(i+1)*config.max_length_s}" for i in range(len(predictions))]

    # DataFrame létrehozása
    pred_df = pd.DataFrame(predictions, columns=[f"species_{i+1}" for i in range(config.num_classes)])
    pred_df.insert(0, "row_id", row_ids)

    # CSV mentése
    pred_df.to_csv(config.output_path, mode="a", index=False, header=not os.path.exists(config.output_path))

In [34]:
# Pipeline
audio_df = list_audio_files(config.data_path)

In [ ]:
for _, row in audio_df.iterrows():
    file_path = row["filepath"]
    filename = row["filepath"].split("/")[-1].replace(".ogg", "")

    # 1. Audio betöltése és szeletelése
    chunks = process_audio(file_path)

    # 2. Mel-spektrogram és inferencia minden szeletre
    predictions = generate_predictions(chunks, model, config)

    # 3. Eredmények mentése CSV-be
    save_predictions_to_csv(filename, predictions)

print(f"Predictions saved to {config.output_path}")